Понимаем сленг с word2vec
======

In [1]:
from gensim.models import Word2Vec
from gensim.matutils import unitvec
import numpy as np
import pandas as pd
from vkml.toolkit import tokenize

Using gpu device 3: GeForce GTX TITAN X (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5005)


Загрузим векторы слов, обученный на корпусе постов и комментариев с открытых стен пабликов. Набор векторов пока публично не доступен.

In [2]:
MODEL_FILENAME = "/d3/models/w2v.model"

In [3]:
MEDUZA_TEST = "./meduza_test.csv"

In [4]:
word_model = Word2Vec.load(MODEL_FILENAME)

Классический тест:

In [5]:
word_model.most_similar(positive=["женщина", "король"], negative=["мужчина"], topn=1)

[('королева', 0.595596194267273)]

In [6]:
word_model.most_similar(positive=["англия", "париж"], negative=["франция"], topn=1)

[('лондон', 0.7713751792907715)]

Попробуем ответить на [тест Медузы про сленг](https://meduza.io/quiz/kek-ili-zatascheno):

In [7]:
df = pd.read_csv(MEDUZA_TEST, encoding="utf-8")

Заменили "затащено" на "затащила", так как "затащено" не оказалось в словаре -____- 

In [8]:
df

,q,a0,a1,a2
0,изи катка,чудовищно,все было просто,"иди к черту, отстань со своими расспросами"
1,приятный,сасный,хэндси,кронги
2,агриться,злиться,cоглашаться,как говорится
3,"я хотел с ней встречаться, но она меня",закрыла,занойзила,отправила во френдзону
4,затащила,на физкультуре одноклассница в одиночку спасла...,одноклассник пришел в старой и немодной одежде.,запущено
5,тян и кун,кое-что не совсем приличное,подтягивания и отжимания,девушка и парень
6,"объясни, почему ты носишь одежду этой марки?",ответь за прикид,поясни за шмот,проясни эмблему


Достаточно среднего вектора слов в предложении:

In [9]:
def get_word_vector(sentence_tokens):
    vectors_available = [word for word in sentence_tokens if word in word_model.vocab]
    result = np.zeros((word_model.vector_size))
    if len(vectors_available) > 0:
        result = np.mean([word_model[word] for word in vectors_available], axis=0)
    return result

Варианты ответов сортирутся по косинусной близости:

In [10]:
def get_sorted_answers(question, answers):
    question_vector = get_word_vector(tokenize(question))
    result = []
    for a in answers:
        answer_vector = get_word_vector(tokenize(a))
        similarity = np.dot(unitvec(answer_vector), unitvec(question_vector))
        result.append((a, similarity))
    return sorted(result, key=lambda x: -x[1])

In [11]:
def print_result(question, sorted_answers):
    print(question)
    for i, (answer, score) in enumerate(sorted_answers):
        print("{}. {:04.2f}: {}".format(i + 1, score, answer))
    print("-" * 20)

In [12]:
for i, test_row in df.iterrows():
    question = test_row["q"]
    answers = [test_row["a{}".format(j)] for j in range(3)]
    sorted_answers = get_sorted_answers(question, answers)
    print_result(question, sorted_answers)

изи катка
1. 0.07: все было просто
2. 0.01: иди к черту, отстань со своими расспросами
3. -0.06: чудовищно
--------------------
приятный
1. 0.35: сасный
2. 0.00: хэндси
3. 0.00: кронги
--------------------
агриться
1. 0.68: злиться
2. 0.11: как говорится
3. 0.00: cоглашаться
--------------------
я хотел с ней встречаться, но она меня
1. 0.16: отправила во френдзону
2. 0.04: закрыла
3. 0.00: занойзила
--------------------
затащила
1. 0.28: на физкультуре одноклассница в одиночку спасла баскетбольный матч.
2. 0.07: одноклассник пришел в старой и немодной одежде.
3. 0.04: запущено
--------------------
тян и кун
1. 0.62: девушка и парень
2. 0.42: кое-что не совсем приличное
3. 0.34: подтягивания и отжимания
--------------------
объясни, почему ты носишь одежду этой марки?
1. 0.33: поясни за шмот
2. 0.30: ответь за прикид
3. 0.16: проясни эмблему
--------------------


Один из вопросов - выбрать неподходящее слово из трех. Легко:

In [13]:
word_model.doesnt_match(["мур", "лол", "кек"])

'мур'

8/8, затащено!